## Udacity Machine Learning - Unsupervised Learning Project Part 2

Working on the recommendation using the PCS( Pearson Correlation Similarity Measure ) and clustering.
This can also be done by nueral network, will try to compare it NN also.
We are working on the MovieLens data set.

### Important Links 
- DATA: https://github.com/udacity/ud741-proj/tree/master/data
- UDACITY COURSE LINK: https://classroom.udacity.com/courses/ud741
- DATA Location: /data
- Helper: movielens.py

#### Approaches
- First Approach : Simple PCS between users, No Clustering, Error evaluation using MSE
- Second Approach : 


In [33]:
%%time 
import numpy as np
from movielens import *
import numpy as np
from sklearn.metrics import mean_squared_error
import math
from tqdm import tnrange
import os.path

CPU times: user 21 µs, sys: 0 ns, total: 21 µs
Wall time: 26 µs


In [2]:
%%time 
# Store data in arrays
user = []
item = []
rating = []
rating_test = []

# Load the movie lens dataset into arrays
d = Dataset()
d.load_users("data/u.user", user)
d.load_items("data/u.item", item)
d.load_ratings("data/u.base", rating)
d.load_ratings("data/u.test", rating_test)

CPU times: user 477 ms, sys: 47.7 ms, total: 525 ms
Wall time: 509 ms


In [3]:
%%time
# The utility matrix stores the rating for each user-item pair in the matrix form.
# Note that the movielens data is indexed starting from 1 (instead of 0).
n_users = len(user)
n_items = len(item)
utility = np.zeros((n_users, n_items))
for r in rating:
    utility[r.user_id-1][r.item_id-1] = r.rating

CPU times: user 55.7 ms, sys: 8.92 ms, total: 64.6 ms
Wall time: 60.3 ms


In [24]:
print 'Number of Users', n_users
print 'Number of Items', n_items

Number of Users 943
Number of Items 1682


In [4]:
%%time
utility = np.zeros((n_users, n_items))
for r in rating:
    utility[r.user_id-1][r.item_id-1] = r.rating

# Finds the average rating for each user and stores it in the user's object
for i in range(n_users):
    rated = np.nonzero(utility[i])
    n = len(rated[0])
    if n != 0:
        user[i].avg_r = np.mean(utility[i][rated])
    else:
        user[i].avg_r = 0.
print utility

[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]
CPU times: user 80.4 ms, sys: 11.6 ms, total: 92.1 ms
Wall time: 85.8 ms


In [48]:
%%time
def pcs(x, y):
    x_y = 0 # numerator
    x_d = 0
    y_d = 0
    counter = 0
    for item_index in range(0, n_items):
        if utility[x.id - 1][item_index] > 0 and utility[y.id - 1][item_index] > 0:
            counter = counter + 1
            x_n = utility[x.id - 1][item_index] - x.avg_r
            y_n = utility[y.id - 1][item_index] - y.avg_r
            x_y = x_y + (x_n * y_n)
            x_d = x_d + (x_n * x_n)    
            y_d = y_d + (y_n * y_n)                               
    if counter > 0 :
        return x_y/(math.sqrt(x_d)* math.sqrt(y_d))
    else:
        return 0

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.11 µs


In [62]:
## create pcs matrix to improve performace
pcs_mat_value = np.zeros((n_users, n_users))
if os.path.isfile('pcs_mat.txt') :
    print 'file present'
else:
    print 'file not present'
    for row in tnrange(0, n_users):
        for col in range(0, n_users):
            if row == col:
                pcs_mat_value[row][col] = -100
            else:
                if col > row :
                    value = pcs(user[row], user[col]);
                    pcs_mat_value[row][col] = value
                else:
                    pcs_mat_value[row][col] = pcs_mat_value[col][row]
                

np.savetxt('pcs_mat.txt', pcs_mat_value, delimiter=',')                 
print pcs_mat_value    

 file not present


HBox(children=(IntProgress(value=0, max=943), HTML(value=u'')))

/Users/amitjain/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


[[-1.00000000e+02  2.63008948e-01 -6.36902086e-02 ...  2.85831102e-01
  -2.51556198e-01  1.53054533e-01]
 [ 2.63008948e-01 -1.00000000e+02 -2.54238425e-01 ... -2.24914353e-01
  -2.53690774e-02  7.18266243e-01]
 [-6.36902086e-02 -2.54238425e-01 -1.00000000e+02 ... -1.00000000e+00
   1.94432904e-01  2.10374129e-01]
 ...
 [ 2.85831102e-01 -2.24914353e-01 -1.00000000e+00 ... -1.00000000e+02
   1.00000000e+00 -1.00000000e+00]
 [-2.51556198e-01 -2.53690774e-02  1.94432904e-01 ...  1.00000000e+00
  -1.00000000e+02  4.45573403e-01]
 [ 1.53054533e-01  7.18266243e-01  2.10374129e-01 ... -1.00000000e+00
   4.45573403e-01 -1.00000000e+02]]


In [63]:
pcs_mat_loaded = np.loadtxt('pcs_mat.txt', delimiter=',')
print pcs_mat_loaded

[[-1.00000000e+02  2.63008948e-01 -6.36902086e-02 ...  2.85831102e-01
  -2.51556198e-01  1.53054533e-01]
 [ 2.63008948e-01 -1.00000000e+02 -2.54238425e-01 ... -2.24914353e-01
  -2.53690774e-02  7.18266243e-01]
 [-6.36902086e-02 -2.54238425e-01 -1.00000000e+02 ... -1.00000000e+00
   1.94432904e-01  2.10374129e-01]
 ...
 [ 2.85831102e-01 -2.24914353e-01 -1.00000000e+00 ... -1.00000000e+02
   1.00000000e+00 -1.00000000e+00]
 [-2.51556198e-01 -2.53690774e-02  1.94432904e-01 ...  1.00000000e+00
  -1.00000000e+02  4.45573403e-01]
 [ 1.53054533e-01  7.18266243e-01  2.10374129e-01 ... -1.00000000e+00
   4.45573403e-01 -1.00000000e+02]]


In [67]:
print len(pcs_mat_loaded), len(pcs_mat_loaded[0])


943 943


In [68]:
def get_n_closet_user(target_user_id, top_n):
    target_user = user[target_user_id-1]
    pcs_array = pcs_mat_loaded[target_user_id -1]
#     for index in range(0, n_users):
#         if target_user_id - 1 == index:
#             pcs_array[target_user_id - 1] = -1
#         else:
#             pcs_array[index - 1] = pcs(target_user, user[index - 1])
    index_of_users = sorted(range(len(pcs_array)), key=lambda i: pcs_array[i])[-top_n:]        
    return np.array(index_of_users) + 1
    

In [69]:
# R(x,i) = Rating of Item I by X
# R(x) = Average Rating given By User X
# get the n top related users, 
# for an item I, get the average of ( R(x,i) - R(x) ) = Delta
# add the Delta to the  average rating of targeted user 
def guess(user_id, i_id, top_n):
#     print 'Guessing the rating for user', user_id,  'of item', i_id
    top_n_user_ids = get_n_closet_user(user_id, top_n);
    delta_sum = 0
    number_of_common_user_rated_i = 0
#     print np.max(top_n_user_ids)
    for index in range(top_n):
        common_user = user[top_n_user_ids[index] - 1]
        if utility[top_n_user_ids[index] - 1][i_id -1] > 0:
            number_of_common_user_rated_i += 1
            delta_sum += utility[top_n_user_ids[index] - 1][i_id -1] - common_user.avg_r
    if number_of_common_user_rated_i == 0:
        return user[user_id-1].avg_r
    delta = delta_sum / number_of_common_user_rated_i
    return user[user_id-1].avg_r + delta

In [78]:
#### calculate mean square error 
print 'start the model traing and error calculation'
n_test = len(rating_test)
print 'Number of tests ', n_test
actual_data = np.zeros(n_test)
estimated_data = np.zeros(n_test)
for index in tnrange(0, n_test): 
    print 'Test case Number ', index
    target_user_id = rating_test[index].user_id
    item_id = rating_test[index].item_id
    estmated_value = guess(target_user_id, item_id, 200)
    estimated_data[index] = estmated_value
    actual_data[index] = rating_test[index].rating
#     print 'actual ', rating_test[index].rating, ' estimated ', estmated_value

MSE = mean_squared_error(estimated_data, actual_data)  
print MSE

start the model traing and error calculation
Number of tests  9430


HBox(children=(IntProgress(value=0, max=9430), HTML(value=u'')))

Test case Number  0
Guessing the rating for user 1 of item 20
Test case Number  1
Guessing the rating for user 1 of item 33
Test case Number  2
Guessing the rating for user 1 of item 61
Test case Number  3
Guessing the rating for user 1 of item 117
Test case Number  4
Guessing the rating for user 1 of item 155
Test case Number  5
Guessing the rating for user 1 of item 160
Test case Number  6
Guessing the rating for user 1 of item 171
Test case Number  7
Guessing the rating for user 1 of item 189
Test case Number  8
Guessing the rating for user 1 of item 202
Test case Number  9
Guessing the rating for user 1 of item 265
Test case Number  10
Guessing the rating for user 2 of item 13
Test case Number  11
Guessing the rating for user 2 of item 50
Test case Number  12
Guessing the rating for user 2 of item 251
Test case Number  13
Guessing the rating for user 2 of item 280
Test case Number  14
Guessing the rating for user 2 of item 281
Test case Number  15
Guessing the rating for user 2 of 

Test case Number  137
Guessing the rating for user 14 of item 474
Test case Number  138
Guessing the rating for user 14 of item 530
Test case Number  139
Guessing the rating for user 14 of item 709
Test case Number  140
Guessing the rating for user 15 of item 25
Test case Number  141
Guessing the rating for user 15 of item 127
Test case Number  142
Guessing the rating for user 15 of item 222
Test case Number  143
Guessing the rating for user 15 of item 331
Test case Number  144
Guessing the rating for user 15 of item 405
Test case Number  145
Guessing the rating for user 15 of item 473
Test case Number  146
Guessing the rating for user 15 of item 678
Test case Number  147
Guessing the rating for user 15 of item 685
Test case Number  148
Guessing the rating for user 15 of item 749
Test case Number  149
Guessing the rating for user 15 of item 932
Test case Number  150
Guessing the rating for user 16 of item 8
Test case Number  151
Guessing the rating for user 16 of item 55
Test case Numb

Guessing the rating for user 27 of item 281
Test case Number  263
Guessing the rating for user 27 of item 298
Test case Number  264
Guessing the rating for user 27 of item 370
Test case Number  265
Guessing the rating for user 27 of item 475
Test case Number  266
Guessing the rating for user 27 of item 596
Test case Number  267
Guessing the rating for user 27 of item 742
Test case Number  268
Guessing the rating for user 27 of item 925
Test case Number  269
Guessing the rating for user 27 of item 978
Test case Number  270
Guessing the rating for user 28 of item 7
Test case Number  271
Guessing the rating for user 28 of item 79
Test case Number  272
Guessing the rating for user 28 of item 95
Test case Number  273
Guessing the rating for user 28 of item 98
Test case Number  274
Guessing the rating for user 28 of item 173
Test case Number  275
Guessing the rating for user 28 of item 201
Test case Number  276
Guessing the rating for user 28 of item 209
Test case Number  277
Guessing the ra

...

Test case Number  8737
Guessing the rating for user 874 of item 321
Test case Number  8738
Guessing the rating for user 874 of item 340
Test case Number  8739
Guessing the rating for user 874 of item 748
Test case Number  8740
Guessing the rating for user 875 of item 133
Test case Number  8741
Guessing the rating for user 875 of item 169
Test case Number  8742
Guessing the rating for user 875 of item 171
Test case Number  8743
Guessing the rating for user 875 of item 268
Test case Number  8744
Guessing the rating for user 875 of item 334
Test case Number  8745
Guessing the rating for user 875 of item 474
Test case Number  8746
Guessing the rating for user 875 of item 514
Test case Number  8747
Guessing the rating for user 875 of item 582
Test case Number  8748
Guessing the rating for user 875 of item 772
Test case Number  8749
Guessing the rating for user 875 of item 923
Test case Number  8750
Guessing the rating for user 876 of item 19
Test case Number  8751
Guessing the rating for us

Test case Number  8903
Guessing the rating for user 891 of item 280
Test case Number  8904
Guessing the rating for user 891 of item 281
Test case Number  8905
Guessing the rating for user 891 of item 471
Test case Number  8906
Guessing the rating for user 891 of item 531
Test case Number  8907
Guessing the rating for user 891 of item 546
Test case Number  8908
Guessing the rating for user 891 of item 591
Test case Number  8909
Guessing the rating for user 891 of item 934
Test case Number  8910
Guessing the rating for user 892 of item 87
Test case Number  8911
Guessing the rating for user 892 of item 134
Test case Number  8912
Guessing the rating for user 892 of item 151
Test case Number  8913
Guessing the rating for user 892 of item 226
Test case Number  8914
Guessing the rating for user 892 of item 273
Test case Number  8915
Guessing the rating for user 892 of item 441
Test case Number  8916
Guessing the rating for user 892 of item 661
Test case Number  8917
Guessing the rating for us

Guessing the rating for user 907 of item 1057
Test case Number  9070
Guessing the rating for user 908 of item 55
Test case Number  9071
Guessing the rating for user 908 of item 56
Test case Number  9072
Guessing the rating for user 908 of item 98
Test case Number  9073
Guessing the rating for user 908 of item 156
Test case Number  9074
Guessing the rating for user 908 of item 173
Test case Number  9075
Guessing the rating for user 908 of item 192
Test case Number  9076
Guessing the rating for user 908 of item 288
Test case Number  9077
Guessing the rating for user 908 of item 414
Test case Number  9078
Guessing the rating for user 908 of item 694
Test case Number  9079
Guessing the rating for user 908 of item 701
Test case Number  9080
Guessing the rating for user 909 of item 116
Test case Number  9081
Guessing the rating for user 909 of item 275
Test case Number  9082
Guessing the rating for user 909 of item 286
Test case Number  9083
Guessing the rating for user 909 of item 292
Test 

Guessing the rating for user 924 of item 195
Test case Number  9236
Guessing the rating for user 924 of item 273
Test case Number  9237
Guessing the rating for user 924 of item 523
Test case Number  9238
Guessing the rating for user 924 of item 1036
Test case Number  9239
Guessing the rating for user 924 of item 1400
Test case Number  9240
Guessing the rating for user 925 of item 5
Test case Number  9241
Guessing the rating for user 925 of item 56
Test case Number  9242
Guessing the rating for user 925 of item 185
Test case Number  9243
Guessing the rating for user 925 of item 218
Test case Number  9244
Guessing the rating for user 925 of item 299
Test case Number  9245
Guessing the rating for user 925 of item 323
Test case Number  9246
Guessing the rating for user 925 of item 332
Test case Number  9247
Guessing the rating for user 925 of item 678
Test case Number  9248
Guessing the rating for user 925 of item 682
Test case Number  9249
Guessing the rating for user 925 of item 948
Test

Guessing the rating for user 941 of item 15
Test case Number  9402
Guessing the rating for user 941 of item 117
Test case Number  9403
Guessing the rating for user 941 of item 124
Test case Number  9404
Guessing the rating for user 941 of item 147
Test case Number  9405
Guessing the rating for user 941 of item 181
Test case Number  9406
Guessing the rating for user 941 of item 257
Test case Number  9407
Guessing the rating for user 941 of item 258
Test case Number  9408
Guessing the rating for user 941 of item 475
Test case Number  9409
Guessing the rating for user 941 of item 993
Test case Number  9410
Guessing the rating for user 942 of item 117
Test case Number  9411
Guessing the rating for user 942 of item 200
Test case Number  9412
Guessing the rating for user 942 of item 261
Test case Number  9413
Guessing the rating for user 942 of item 323
Test case Number  9414
Guessing the rating for user 942 of item 423
Test case Number  9415
Guessing the rating for user 942 of item 427
Test

In [79]:
print MSE

0.9732460255725063


In [82]:
round(,4)

2417704920.63194

In [84]:
for i in range(n_test):
    print rating_test[i].user_id,',',rating_test[i].item_id,',',round(estimated_data[i], 4)

1 , 20 , 3.8589
1 , 33 , 3.4965
1 , 61 , 4.3713
1 , 117 , 3.722
1 , 155 , 2.3962
1 , 160 , 4.0416
1 , 171 , 4.4286
1 , 189 , 4.2998
1 , 202 , 4.1128
1 , 265 , 3.8979
2 , 13 , 3.6792
2 , 50 , 4.4635
2 , 251 , 4.2264
2 , 280 , 3.5339
2 , 281 , 3.4536
2 , 290 , 3.2519
2 , 292 , 3.9151
2 , 297 , 4.2723
2 , 312 , 3.4686
2 , 314 , 3.75
3 , 245 , 2.6435
3 , 294 , 2.5602
3 , 323 , 2.2597
3 , 328 , 3.0088
3 , 331 , 2.8635
3 , 332 , 3.0793
3 , 334 , 3.4469
3 , 335 , 4.1705
3 , 337 , 2.6782
3 , 343 , 2.384
4 , 50 , 5.1127
4 , 260 , 3.6901
4 , 264 , 3.6123
4 , 288 , 4.1581
4 , 294 , 4.0714
4 , 303 , 4.5559
4 , 354 , 4.0743
4 , 356 , 3.9533
4 , 357 , 5.1639
4 , 361 , 4.7144
5 , 1 , 3.2891
5 , 2 , 2.547
5 , 17 , 2.7078
5 , 98 , 3.5932
5 , 110 , 2.4084
5 , 225 , 2.3674
5 , 363 , 2.0688
5 , 424 , 1.4593
5 , 439 , 2.9091
5 , 454 , 3.0779
6 , 14 , 3.9826
6 , 23 , 4.3103
6 , 69 , 3.6525
6 , 86 , 3.7827
6 , 98 , 4.3494
6 , 258 , 3.6699
6 , 301 , 3.2126
6 , 463 , 3.7703
6 , 492 , 4.1306
6 , 517 , 4.095
7 ,

65 , 806 , 3.1346
65 , 1142 , 4.6227
66 , 1 , 3.8916
66 , 7 , 3.721
66 , 117 , 3.4897
66 , 181 , 4.0464
66 , 249 , 2.9071
66 , 258 , 3.7127
66 , 298 , 3.6527
66 , 471 , 3.3381
66 , 597 , 3.17
66 , 877 , 3.3047
67 , 1 , 3.8669
67 , 64 , 4.4141
67 , 121 , 3.5099
67 , 147 , 3.4268
67 , 151 , 3.5001
67 , 240 , 2.7349
67 , 405 , 3.3797
67 , 743 , 2.4835
67 , 1093 , 3.628
67 , 1095 , 2.1981
68 , 7 , 3.4279
68 , 117 , 3.0426
68 , 118 , 2.4602
68 , 178 , 3.851
68 , 237 , 3.3159
68 , 245 , 2.7865
68 , 405 , 2.6741
68 , 411 , 2.4608
68 , 742 , 3.0535
68 , 1047 , 2.0938
69 , 12 , 4.7376
69 , 56 , 4.2366
69 , 174 , 4.5894
69 , 175 , 3.9033
69 , 240 , 3.2731
69 , 256 , 3.8112
69 , 258 , 4.0841
69 , 268 , 4.3825
69 , 300 , 3.6793
69 , 427 , 4.4999
70 , 50 , 4.2576
70 , 82 , 3.6099
70 , 210 , 3.6348
70 , 217 , 3.2379
70 , 399 , 3.0775
70 , 419 , 3.6251
70 , 431 , 3.789
70 , 584 , 3.5436
70 , 746 , 3.6153
70 , 1133 , 2.8562
71 , 56 , 3.9594
71 , 89 , 4.2426
71 , 134 , 4.3555
71 , 222 , 3.738
71 , 276 

125 , 478 , 3.9909
125 , 657 , 4.0833
125 , 997 , 2.971
126 , 245 , 3.3246
126 , 258 , 3.8775
126 , 262 , 3.7618
126 , 294 , 3.2491
126 , 322 , 3.1797
126 , 323 , 2.992
126 , 332 , 3.5922
126 , 878 , 2.3098
126 , 884 , 3.5714
126 , 905 , 3.5637
127 , 227 , 3.8284
127 , 229 , 3.9085
127 , 243 , 3.1115
127 , 268 , 4.4324
127 , 286 , 4.6488
127 , 294 , 4.0881
127 , 300 , 4.364
127 , 380 , 4.2882
127 , 449 , 3.6605
127 , 748 , 4.0448
128 , 15 , 3.6999
128 , 182 , 4.2812
128 , 204 , 3.9338
128 , 317 , 3.9781
128 , 371 , 3.1615
128 , 482 , 4.072
128 , 485 , 3.9628
128 , 508 , 3.6693
128 , 603 , 4.3503
128 , 739 , 3.5239
129 , 269 , 3.0139
129 , 270 , 2.8066
129 , 272 , 3.4466
129 , 303 , 2.849
129 , 307 , 2.6762
129 , 313 , 3.2168
129 , 331 , 2.5139
129 , 339 , 2.265
129 , 873 , 2.1363
129 , 995 , 2.5954
130 , 105 , 2.8219
130 , 109 , 3.9193
130 , 216 , 4.2262
130 , 379 , 3.3378
130 , 642 , 3.8575
130 , 1014 , 3.6935
130 , 1017 , 3.9207
130 , 1217 , 3.5194
130 , 1246 , 1.6828
130 , 1248 , 3.

172 , 603 , 3.7645
172 , 657 , 3.7044
172 , 697 , 3.3222
173 , 242 , 4.4451
173 , 260 , 3.379
173 , 286 , 4.3411
173 , 289 , 3.7574
173 , 294 , 3.7768
173 , 300 , 4.3539
173 , 322 , 3.6845
173 , 323 , 3.6884
173 , 328 , 4.211
173 , 938 , 4.1458
174 , 80 , 2.9584
174 , 160 , 3.7253
174 , 237 , 3.7545
174 , 315 , 4.3641
174 , 368 , 1.4459
174 , 395 , 2.0156
174 , 402 , 3.5829
174 , 471 , 3.7411
174 , 660 , 4.6114
174 , 937 , 2.8814
175 , 31 , 3.9591
175 , 56 , 4.0588
175 , 71 , 4.193
175 , 96 , 4.3031
175 , 127 , 4.4893
175 , 176 , 4.2354
175 , 183 , 4.0812
175 , 483 , 4.5453
175 , 629 , 3.4579
175 , 669 , 2.6433
176 , 258 , 3.9574
176 , 272 , 4.3761
176 , 288 , 3.5848
176 , 294 , 3.1945
176 , 297 , 4.1615
176 , 305 , 3.8156
176 , 324 , 3.0582
176 , 327 , 3.8884
176 , 751 , 3.5173
176 , 875 , 3.3277
177 , 56 , 4.369
177 , 69 , 3.8227
177 , 79 , 4.0601
177 , 203 , 4.2043
177 , 209 , 4.2965
177 , 243 , 2.0401
177 , 299 , 2.7094
177 , 321 , 3.4052
177 , 333 , 3.658
177 , 960 , 4.0799
178 , 

224 , 26 , 2.4727
224 , 29 , 2.3231
224 , 69 , 3.4733
224 , 77 , 3.0302
224 , 178 , 3.3392
224 , 191 , 3.4808
224 , 245 , 2.892
224 , 583 , 2.6283
224 , 678 , 2.4835
224 , 873 , 2.9832
225 , 64 , 5.6024
225 , 136 , 5.3259
225 , 143 , 5.0935
225 , 172 , 5.3547
225 , 193 , 5.0961
225 , 237 , 4.8305
225 , 418 , 4.6614
225 , 480 , 5.3434
225 , 492 , 4.6386
225 , 1443 , 4.6471
226 , 7 , 4.1752
226 , 14 , 4.044
226 , 56 , 4.0721
226 , 147 , 3.6321
226 , 169 , 4.8591
226 , 180 , 4.1718
226 , 242 , 4.0807
226 , 270 , 4.0082
226 , 283 , 3.9308
226 , 480 , 4.5057
227 , 9 , 4.0629
227 , 50 , 4.0834
227 , 117 , 3.3261
227 , 126 , 3.441
227 , 244 , 3.5752
227 , 276 , 3.7027
227 , 286 , 3.6909
227 , 293 , 3.5847
227 , 295 , 2.8447
227 , 411 , 2.2862
228 , 87 , 3.6164
228 , 204 , 3.366
228 , 288 , 2.9136
228 , 313 , 3.702
228 , 427 , 3.7172
228 , 475 , 3.3288
228 , 650 , 3.1401
228 , 651 , 3.473
228 , 655 , 3.3512
228 , 886 , 2.7656
229 , 260 , 2.3344
229 , 286 , 3.4531
229 , 313 , 3.8256
229 , 315 ,

...

694 , 196 , 4.588
694 , 229 , 3.7524
694 , 300 , 4.2033
694 , 419 , 4.297
694 , 432 , 4.4459
694 , 684 , 4.3339
694 , 1050 , 4.6177
695 , 260 , 2.459
695 , 300 , 3.2623
695 , 301 , 3.0138
695 , 307 , 3.3954
695 , 313 , 3.9655
695 , 319 , 3.1392
695 , 338 , 2.4063
695 , 882 , 3.5828
695 , 995 , 2.8355
695 , 1024 , 3.8098
696 , 234 , 4.0281
696 , 305 , 4.0826
696 , 310 , 3.7035
696 , 312 , 3.2774
696 , 315 , 4.4808
696 , 344 , 3.1893
696 , 523 , 4.3605
696 , 689 , 3.0727
696 , 883 , 4.6264
696 , 1176 , 4.9494
697 , 150 , 3.935
697 , 273 , 3.5732
697 , 294 , 3.4702
697 , 298 , 4.0168
697 , 301 , 3.2701
697 , 302 , 4.3798
697 , 331 , 3.774
697 , 333 , 3.6379
697 , 683 , 2.5288
697 , 713 , 3.745
698 , 50 , 3.7286
698 , 83 , 3.4163
698 , 95 , 3.2321
698 , 168 , 3.3989
698 , 478 , 3.2545
698 , 497 , 2.8389
698 , 512 , 2.6095
698 , 529 , 3.2809
698 , 568 , 2.9899
698 , 1063 , 2.4912
699 , 1 , 3.2243
699 , 16 , 2.9256
699 , 100 , 3.8027
699 , 321 , 2.7622
699 , 324 , 3.1977
699 , 456 , 2.2281
6

755 , 302 , 3.6559
755 , 304 , 3.0292
755 , 319 , 2.6861
755 , 343 , 2.642
755 , 689 , 2.3481
755 , 690 , 3.1067
755 , 875 , 2.3236
756 , 63 , 3.0244
756 , 99 , 3.1035
756 , 117 , 3.247
756 , 178 , 3.893
756 , 367 , 3.1952
756 , 398 , 2.1531
756 , 404 , 3.0456
756 , 527 , 3.5051
756 , 550 , 2.8977
756 , 603 , 3.8756
757 , 7 , 3.7534
757 , 71 , 3.5318
757 , 100 , 4.0778
757 , 161 , 3.4252
757 , 196 , 3.6956
757 , 226 , 3.3969
757 , 399 , 3.4163
757 , 569 , 2.4574
757 , 651 , 3.9164
757 , 771 , 2.963
758 , 127 , 4.8123
758 , 135 , 4.5897
758 , 235 , 3.3359
758 , 258 , 4.0943
758 , 272 , 4.5921
758 , 298 , 4.2468
758 , 347 , 3.7233
758 , 517 , 4.1957
758 , 587 , 3.8606
758 , 597 , 3.2746
759 , 1 , 4.2364
759 , 117 , 3.9926
759 , 121 , 3.8493
759 , 258 , 4.3592
759 , 298 , 4.249
759 , 300 , 4.041
759 , 332 , 3.8995
759 , 678 , 3.5277
759 , 742 , 3.9965
759 , 1016 , 3.8006
760 , 65 , 3.2143
760 , 181 , 3.9013
760 , 183 , 3.7956
760 , 185 , 3.9634
760 , 202 , 3.7831
760 , 216 , 3.8052
760 , 

802 , 447 , 3.3258
802 , 452 , 2.494
803 , 243 , 2.2671
803 , 245 , 2.8841
803 , 271 , 2.9126
803 , 286 , 3.2191
803 , 311 , 3.478
803 , 688 , 2.1903
803 , 690 , 2.9255
803 , 748 , 2.8252
803 , 754 , 2.7177
803 , 887 , 3.2869
804 , 11 , 4.0739
804 , 31 , 4.0453
804 , 82 , 4.0281
804 , 198 , 4.3436
804 , 204 , 4.288
804 , 318 , 4.5159
804 , 399 , 3.5699
804 , 546 , 3.1985
804 , 588 , 3.8718
804 , 1076 , 2.6073
805 , 17 , 2.7422
805 , 176 , 3.7617
805 , 319 , 3.088
805 , 321 , 3.0397
805 , 387 , 3.0998
805 , 475 , 3.7945
805 , 595 , 3.0084
805 , 715 , 3.3709
805 , 747 , 2.4235
805 , 755 , 3.5593
806 , 3 , 3.1065
806 , 177 , 4.1969
806 , 182 , 4.2978
806 , 192 , 4.3439
806 , 324 , 3.2664
806 , 405 , 3.4438
806 , 407 , 2.349
806 , 629 , 3.52
806 , 654 , 4.4662
806 , 705 , 4.1654
807 , 117 , 4.0065
807 , 239 , 4.1838
807 , 250 , 3.9991
807 , 385 , 3.9777
807 , 399 , 3.7476
807 , 419 , 4.3252
807 , 465 , 3.8024
807 , 471 , 3.7755
807 , 636 , 3.5478
807 , 1444 , 4.0365
808 , 245 , 3.734
808 ,

859 , 1061 , 3.1024
859 , 1095 , 3.2568
860 , 4 , 3.1562
860 , 159 , 2.9466
860 , 211 , 3.8043
860 , 283 , 3.2238
860 , 289 , 2.6016
860 , 294 , 2.6457
860 , 303 , 3.3922
860 , 514 , 3.6382
860 , 715 , 3.5084
860 , 1061 , 2.7169
861 , 52 , 3.7958
861 , 86 , 4.0482
861 , 170 , 4.1319
861 , 275 , 4.2591
861 , 289 , 3.4881
861 , 294 , 3.6204
861 , 714 , 3.8788
861 , 736 , 3.8431
861 , 949 , 3.3566
861 , 1227 , 2.9524
862 , 50 , 5.3494
862 , 127 , 5.0138
862 , 168 , 5.1087
862 , 193 , 4.6814
862 , 210 , 4.7712
862 , 215 , 4.4343
862 , 288 , 4.1989
862 , 357 , 5.1425
862 , 407 , 3.3753
862 , 505 , 4.9107
863 , 264 , 2.4646
863 , 270 , 3.3367
863 , 301 , 2.9587
863 , 333 , 3.3895
863 , 352 , 2.3235
863 , 754 , 2.9277
863 , 882 , 3.3491
863 , 902 , 3.5106
863 , 1062 , 2.9389
863 , 1294 , 3.4531
864 , 50 , 4.8845
864 , 53 , 3.424
864 , 71 , 4.0716
864 , 168 , 4.4352
864 , 208 , 4.2992
864 , 273 , 3.9311
864 , 408 , 4.919
864 , 642 , 3.6183
864 , 708 , 3.581
864 , 1446 , 4.3986
865 , 7 , 2.5257

905 , 245 , 3.0608
905 , 258 , 3.4835
905 , 282 , 3.3688
905 , 300 , 3.4306
905 , 321 , 3.3952
905 , 328 , 3.158
905 , 333 , 3.3585
905 , 717 , 2.6679
906 , 7 , 4.0479
906 , 10 , 4.324
906 , 100 , 4.3239
906 , 121 , 3.46
906 , 307 , 3.8957
906 , 321 , 3.4538
906 , 475 , 4.3487
906 , 628 , 3.7479
906 , 696 , 3.2026
906 , 740 , 3.8061
907 , 100 , 5.0708
907 , 290 , 3.9223
907 , 317 , 4.9988
907 , 340 , 4.6106
907 , 591 , 4.593
907 , 619 , 4.3653
907 , 685 , 4.607
907 , 724 , 4.5895
907 , 763 , 4.7355
907 , 1057 , 4.6058
908 , 55 , 3.1354
908 , 56 , 3.8566
908 , 98 , 4.0681
908 , 156 , 3.4698
908 , 173 , 3.9992
908 , 192 , 4.119
908 , 288 , 3.4919
908 , 414 , 3.2033
908 , 694 , 3.6382
908 , 701 , 3.7986
909 , 116 , 4.6087
909 , 275 , 4.7728
909 , 286 , 4.5261
909 , 292 , 4.0308
909 , 300 , 4.4106
909 , 509 , 4.277
909 , 531 , 4.4868
909 , 582 , 3.9997
909 , 682 , 3.2162
909 , 1121 , 4.485
910 , 23 , 3.7358
910 , 100 , 3.5644
910 , 117 , 2.9245
910 , 182 , 3.7012
910 , 237 , 3.3116
910 , 2

In [85]:
len(estimated_data)

9430